# TODO / Commentaire
 - Est ce qu'on met en log les births ?
 - 
 
 use of contraceptive method
 abortion
 impact of emmigration
 children as workpower

# Wesh les reufs bien ou quoi

Importing data:
 - `valeurs_mensuelles.csv`: containing the number of birth in France for each month from 01-01-2004 to 01-10-2020.<br>
 Columns:
     - Période: month
     - Démographie - Nombre de naissances vivantes - France métropolitaine: number of birth for the corresponding month
     - Codes: Codes for explaining what type of value we have in our case we have "P" for 2020 data standing for "provisional" and "A" everywhere else standing for normale value. (see `caract.csv` for further details)
 - `GTD.csv`: containing the query number of google trend data for different categories.<br>
   This dataframe is created in the notebook `GTD_preprocessing.ipynb`.<br>
 Columns:
     - date: date of the query number
     - category id : columns containing all the query number in France for this category.
     
 - `categories.csv`: containing all the ids and names of the google trend categories<br>
   This dataframe is created in the notebook `GTD_preprocessing.ipynb`. The categories can be seen as a list with sub-categories here: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories<br>
 Columns:
     - id: id of the diffrent category
     - name: Name of this category

In [1]:
#***Attention***  les donnees n'ont pas les memes tailles
#valeurs mensuelle = 202x1        01-01-2004  --> 01-10-2020 
#GTD               = 204×916      01-01-2004  --> 01-12-2020       916 categories       (donnée incomplete pour decembre)    (certaines categories ne donne pas de resultat exemple : category 42 "jazz" https://trends.google.com/trends/explore?cat=42&date=all&geo=FR )
# categories       = 1133x3                                       1133 categories       (il y a bien 1133 category differentes sur google trend mais dans GTD on a enlevé les category sans resultat  il en reste 916)

In [2]:
# Clear all variables
#%reset -f

# Importing librairies: numpy, matplotlib, pandas, statmodel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf



In [3]:
# Importing birth data with selecting correct columns and changing columns names
data_folder = './Data/'
birth_df = pd.read_csv(data_folder +'valeurs_mensuelles.csv', sep= ';', index_col= 0, skiprows=2, usecols=[0,1],header=0, names=['Période','births'])

# Changing indexes to panda date
birth_df.index = pd.to_datetime(birth_df.index)

# Putting sales data in log scale as we want
birth_df['births'] = birth_df['births'].apply(np.log)

# Getting an overview
birth_df.sample(5)

,births
Période,
2009-04-01,11.059740
2015-01-01,11.075908
2010-02-01,11.020382
2010-09-01,11.126660
2011-05-01,11.121718


In [4]:
# Importing google trend data
GTD_df = pd.read_csv(data_folder +'GTD.csv',index_col= 0)

# Getting an overview
GTD_df.sample(5, axis=1)

,1076,112,671,1385,1013
date,,,,,
2004-01-01,82,28,35,67,77
2004-02-01,75,29,35,100,76
2004-03-01,78,29,37,51,77
2004-04-01,82,28,38,88,76
2004-05-01,86,32,40,14,73
...,...,...,...,...,...
2020-08-01,59,51,61,9,51
2020-09-01,61,55,59,14,49
2020-10-01,59,63,54,11,47


In [5]:
# Importing google trend categories
categories_df = pd.read_csv(data_folder +'categories.csv',index_col= 0)

# Getting an overview
categories_df.sample(5)

,id,name
1212,436,Mathématiques
328,582,Blogs et sites personnels
158,1040,Blues
794,1313,Politique d'immigration et problèmes frontaliers
121,1099,Westerns


## Analyse des categories

In [6]:
# Temporaire
# PETITE CORRELATION TOUT SIMPLE

temp2 = []
for i,j in enumerate(categories_df['id']):
    if str(j) in GTD_df.columns:
        temp = np.abs(birth_df.iloc[:,0].corr(GTD_df[str(j)],method='pearson'))
        temp2.append(temp)
    else:
        temp2.append(np.nan)
    
categories_df['correlation'] = temp2
categories_df.sort_values('correlation', ascending=False)

,id,name,correlation
1126,1211,ORL,0.697831
1137,1238,Nettoyage du corps et désintoxication,0.690711
1145,645,Pédiatrie,0.660431
1184,245,Soins bucco-dentaires,0.656407
1348,557,Massothérapie,0.637538
...,...,...,...
1255,1384,"Jumelles, télescopes et appareils optiques",NaN
612,1393,Décodeurs et magnétoscopes numériques,NaN
614,1394,Lecteurs et enregistreurs Blu-Ray,NaN
615,1395,Lecteurs et enregistreurs DVD,NaN
